In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("lastfm.csv")

### Для анализа выберем Японию и США

In [ ]:
japan_data = data.loc[data["country"] == 'Japan']
usa_data = data.loc[data["country"] == 'United States']
options = ['Japan', 'United States']
true_data = data.loc[data["country"].isin(options)]
true_data

In [ ]:
true_data.artist.value_counts().sort_values(ascending = False)

### Подготовка данных для ассоциативного анализа

#### Выполним группировки

In [ ]:
#по пользователю, можно удалить
groupped_user = true_data.groupby('user')['artist'].apply(';'.join)
groupped_user

#### Выполним бинаризацию

In [ ]:
#по пользователю, можно удалить
binary_data_u = groupped_user.str.get_dummies(";")
binary_data_u.iloc[5:15, 12:23]

In [ ]:
pivoted_artist = true_data[['user', 'artist']].pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)
pivoted_artist_jp = japan_data[['user', 'artist']].pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)
pivoted_artist_usa = usa_data[['user', 'artist']].pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)
pivoted_country = true_data[['user', 'country']].pivot_table(index='user', columns='country',aggfunc=any, fill_value=False).astype(int)
pivoted_sex_jp = japan_data[['user', 'sex']].pivot_table(index='user', columns='sex',aggfunc=any, fill_value=False).astype(int)
pivoted_sex_usa = usa_data[['user', 'sex']].pivot_table(index='user', columns='sex',aggfunc=any, fill_value=False).astype(int)
#pivoted_all = pd.concat([pivoted_artist, pivoted_country, pivoted_sex], axis=1)

In [ ]:
pivoted_all_japan = pd.concat([pivoted_artist_jp, pivoted_sex_jp], axis=1)
pivoted_all_japan

In [ ]:
pivoted_all_usa = pd.concat([pivoted_artist_usa, pivoted_sex_usa], axis=1)
pivoted_all_usa

In [ ]:
pivoted_all_country = pd.concat([pivoted_artist, pivoted_country], axis=1)
pivoted_all_country

### Поиск характерных комбинаций

In [ ]:
import mlxtend.frequent_patterns as ml

In [ ]:
#по пользователю, можно удалить
freq_items_u = ml.apriori(binary_data_u, min_support = 0.04, use_colnames = True)
print("Найдено {0} характерных комбинаций".format(len(freq_items_u)))
freq_items_u

In [ ]:
#по пользователю, можно удалить
rules_u = ml.association_rules(freq_items_u, metric = "confidence", min_threshold = 0.4)
rules_u.sort_values('confidence', ascending=False)

In [ ]:
apriori_sex_jp = ml.apriori(pivoted_all_japan, min_support=0.03, use_colnames=True)
apriori_sex_jp

In [ ]:
apriori_sex_jp_rules = ml.association_rules(apriori_sex_jp, metric="confidence", min_threshold=0.4)
apriori_sex_jp_rules.sort_values('confidence', ascending=False)

In [ ]:
apriori_sex_usa = ml.apriori(pivoted_all_usa, min_support=0.05, use_colnames=True)
apriori_sex_usa

In [ ]:
apriori_sex_usa_rules = ml.association_rules(apriori_sex_usa, metric="confidence", min_threshold=0.8)
apriori_sex_usa_rules.sort_values('confidence', ascending=False)